In [1]:
#treadmill_task_data_dissection.ipynb

In [2]:
#trial configuration

#Reward Zone
RZceil = 170 
RZfloor = 150

drop_start_licks_before = 20 #before position(cm)

In [12]:
#load data file
import os
import pandas as pd
from tkinter import Tk, filedialog

# 選取多個資料檔案
root = Tk()
root.attributes('-topmost', True)  # 將主視窗設置為最上層
root.withdraw()

data_file_paths = filedialog.askopenfilenames(parent = root ,title='選擇資料檔案', filetypes=[('CSV files', '*.csv')])

all_raw_df = {}


# 循環處理每個選取的檔案
for index, data_file_dir in enumerate(data_file_paths):
    data_file_name = os.path.basename(data_file_dir)
    file_name_parts = data_file_name.split('_')
    mouseID = file_name_parts[0]
    Date = file_name_parts[1]
    schedule = file_name_parts[2]

    print("當前處理檔案:" + data_file_name)

    df = pd.read_csv(data_file_dir)
    if df.columns[0] == "Trial_number":
        print("檔案格式正確")
    else:
        print("檔案格式錯誤")

    all_raw_df[index] = df


當前處理檔案:DoB0710a_0810_36CTZ_150cm_12ul.csv
檔案格式正確


In [4]:
#提取某些資訊


#(函數)區分Trial成功失敗
def Trial_success_and_fail(all_raw_df, index):
    raw_df = all_raw_df[index]
    raw_df = raw_df.dropna()
    trial_df = raw_df.groupby("Trial_number")
    success_trial = pd.DataFrame([])
    fail_trial = pd.DataFrame([])

    for trial_number, group_data in trial_df:

        #找RZ裡的licking
        data_with_licking = group_data[group_data["Licking_count"].diff() != 0] 
        data_with_licking_RZfloor = data_with_licking[data_with_licking["Distance"] >= RZfloor]
        data_with_licking_RZfloor_RZceil = data_with_licking_RZfloor[data_with_licking_RZfloor["Distance"] <= RZceil]

        #如果RZ裡有licking
        if (len(data_with_licking_RZfloor_RZceil) > 0):
            success_trial = pd.concat([success_trial, group_data], axis=0, ignore_index=True)
        else:
            fail_trial = pd.concat([fail_trial, group_data], axis=0, ignore_index=True)

    return success_trial, fail_trial



#(函數)Trial_number的起始時間

def get_start_time_of_trial(index, trial_number):

    raw_df = all_raw_df[index]
    raw_df = raw_df.dropna()
    trial_start_time = raw_df[raw_df["Trial_number"] == trial_number]["Time"].iloc[0]

    return trial_start_time



#(函數)找到是否注射及其位置
def injection_trial(data_file_dir):
    data_file_name = os.path.basename(data_file_dir)
    file_name_parts = data_file_name.split('_')
    mouseID = file_name_parts[0]
    Date = file_name_parts[1]
    schedule = file_name_parts[2]
    if "CTZ" in schedule or "saline" in schedule:
        injection = True
        injection_trial = schedule[:2]
        
    else:
        injection = False
        injection_trial = np.nan
        
    return injection, injection_trial

In [5]:
#(函數)計算每個Trial中licking event發生位置的mean, var, std
def licking_statistic(all_raw_df, index):
    raw_df = all_raw_df[index]
    raw_df = raw_df.dropna()
    trial_df = raw_df.groupby("Trial_number")
    All_trial_licks = {}

    for trial_number, group_data in trial_df:

        # 找出 Licking_count 出現變化的行
        Licking_rows = group_data[group_data["Licking_count"].diff() != 0]
        Licking_rows = Licking_rows.drop(Licking_rows.index[0])
        if (len(Licking_rows.loc[Licking_rows['Distance'] >= 150])-len(Licking_rows.loc[Licking_rows['Distance'] >= 170])) > 0:
            first_lick_in_RZ = Licking_rows.loc[Licking_rows['Distance'] >= 150].iloc[0]
            first_lick_in_RZ = pd.DataFrame(first_lick_in_RZ)
            first_lick_in_RZ = first_lick_in_RZ.T
            Licking_rows = Licking_rows.loc[group_data["Distance"] < RZfloor]
            Licking_rows = Licking_rows.loc[group_data["Distance"] > drop_start_licks_before]
            Licking_rows = pd.concat([Licking_rows, first_lick_in_RZ], axis=0, ignore_index=True)
            trial_success = True

        else :
            Licking_rows = Licking_rows.loc[group_data["Distance"] < RZfloor]
            Licking_rows = Licking_rows.loc[group_data["Distance"] > drop_start_licks_before]
            trial_success = False
        
        if len(Licking_rows) == 0:
            lick_mean = pd.NA
            lick_var = pd.NA
            lick_std = pd.NA

        elif len(Licking_rows) == 1:
            lick_mean = 150
            lick_var = 0
            lick_std = 0
            
        else:
            lick_mean = round(Licking_rows["Distance"].mean(), 2)
            lick_var = round(Licking_rows["Distance"].var(), 2)
            lick_std = round(Licking_rows["Distance"].std(), 2)

        All_trial_licks[int(trial_number)] = {"success":trial_success, "mean": lick_mean,"var": lick_var,"std": lick_std}

    All_trial_licks = pd.DataFrame(All_trial_licks)
    All_trial_licks = All_trial_licks.T
    All_trial_licks = All_trial_licks.sort_index()

    return All_trial_licks


In [6]:
#輸出多檔案licking_statistic結果
excel_writer = pd.ExcelWriter('output.xlsx')
for index, data_file_dir in enumerate(data_file_paths):
    
    data_file_name = os.path.basename(data_file_dir)
    print("當前處理檔案:" + data_file_name)
    file_name_parts = data_file_name.split('_')
    mouseID = file_name_parts[0]
    Date = file_name_parts[1]
    schedule = file_name_parts[2]

    licking_statistic_df = licking_statistic(all_raw_df, index)
    
    
    licking_statistic_df.to_excel(excel_writer, sheet_name = mouseID + "_" + Date+ "_" + schedule, index=True)

# 關閉 ExcelWriter，將資料寫入檔案中
excel_writer.save()

當前處理檔案:G270_0718_stage3_150cm_12ul_2.csv
當前處理檔案:G270_0720_stage3_150cm_12ul.csv


In [13]:
#behavior trace plot
import matplotlib.pyplot as plt
import numpy as np

for index, data_file_dir in enumerate(data_file_paths):

    #讀取資料
    data_file_name = os.path.basename(data_file_dir)
    df = all_raw_df[index]
    trial_df = df.groupby("Trial_number")

    #視覺化behavior trace
    fig, ax = plt.subplots(figsize=(10, 6), dpi=1200)

    # 設置RZ底色
    ax.axhspan(RZfloor, RZceil, facecolor='cyan', label='reward zone')

    # 繪製RZ界線
    ax.axhline(y=RZfloor, color='blue', linestyle='dashed', linewidth=1)
    ax.axhline(y=RZceil, color='blue', linestyle='dashed', linewidth=1)

    # 繪製移動軌跡
    success_df, fail_df = Trial_success_and_fail(all_raw_df, index)
    if(len(success_df) > 0):
        plt.scatter(success_df['Time'] / (1000 * 60), success_df['Distance'], color='lightgreen', s=1, label='success', marker = '_')
    if(len(fail_df) > 0):
        plt.scatter(fail_df['Time'] / (1000 * 60), fail_df['Distance'], color='lightcoral', s=1, label='fail', marker = '_')

    # 繪製licking
    licking_points = np.where(df['Licking_count'].diff() != 0)[0]
    plt.scatter(df['Time'].iloc[licking_points] / (1000 * 60), df['Distance'].iloc[licking_points], color='darkblue', s=2, label='Licking', zorder=10)

    # 繪製注射時間點
    injection, trial_injected = injection_trial(data_file_dir)
    
    if injection == True:
        injection_trial_time = get_start_time_of_trial(index, trial_injected)
        ax.axvline(x = injection_trial_time / (1000 * 60), color='gray', linestyle='dashed', linewidth=1)

    # 設置圖形的原點位置
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')

    # 圖例
    plt.legend(scatterpoints=5, loc='upper right', prop={'size': 6})

    # 坐標軸標題
    plt.xlabel('Time (minutes)')
    plt.ylabel('Distance (cm)')

    # 設置軸範圍
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 200)


    # 圖表標題
    plt.title('Plot of Distance vs Time\nFile: {}'.format(data_file_name))
